**Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


**Import required modules as well**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df= pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Tips.csv')

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [ ]:
df.shape

(244, 7)

**Define X & y variables**

In [ ]:
df.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

In [ ]:
X= df[['tip', 'sex', 'smoker', 'day', 'time', 'size']]
y= df['total_bill']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X,y, random_state=42, test_size=0.2)

**Pipeline**

**Imputation of missing value and Scaling**

In [ ]:
numeric_preprocessor= Pipeline(
    steps=[
        ("imputation_mean",SimpleImputer(missing_values= np.nan, strategy='mean')),
        ('scaler', StandardScaler()),

     ])

In [ ]:
categorical_preprocessor= Pipeline(
    steps=[
        ("imputation_constant",SimpleImputer(fill_value= 'missing', strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore')),

    ])

In [ ]:
preprocessor= ColumnTransformer([
    ('categorical',categorical_preprocessor,['sex', 'smoker', 'day', 'time']),
    ('numerical', numeric_preprocessor,['tip', 'size'])

])

In [ ]:
pipe= Pipeline(
    steps=[("Preprocessor",preprocessor),
           ('regressor', RandomForestRegressor())
       ])

In [ ]:
from sklearn import set_config

In [ ]:
set_config(display='diagram')

In [ ]:
pipe

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor', RandomForestRegressor())])

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor', RandomForestRegressor())])

In [ ]:
y_pred=pipe.predict(X_test)

**Hyper Tuning**

In [ ]:
param_grid= {
    "regressor__n_estimators": [200,500],
    "regressor__max_features": ["auto","sqrt","log2"],
    "regressor__max_depth": [4,5,6,7,8]
}

In [ ]:
grid_search= GridSearchCV(pipe, param_grid=param_grid, n_jobs=1)

In [ ]:
grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('Preprocessor',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('imputation_constant',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('onehot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['sex',
                                                                          'smoker',
                                                                          'day',
                                                                          'time']),
                                                                        ('numerical',
                                                                         Pipeline(steps=[('imputation_mean',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['tip',
                                                                          'size'])])),
                                       ('regressor', RandomForestRegressor())]),
             n_jobs=1,
             param_grid={'regressor__max_depth': [4, 5, 6, 7, 8],
                         'regressor__max_features': ['auto', 'sqrt', 'log2'],
                         'regressor__n_estimators': [200, 500]})

In [ ]:
grid_search.best_params_

{'regressor__max_depth': 6,
 'regressor__max_features': 'sqrt',
 'regressor__n_estimators': 500}

**The best parameters are this, hence we take this and use**

In [ ]:
pipe= Pipeline(
    steps=[("Preprocessor",preprocessor),
           ('regressor', RandomForestRegressor(max_depth=6, max_features='sqrt', n_estimators=500))
       ])

In [ ]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['sex', 'smoker', 'day',
                                                   'time']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['tip', 'size'])])),
                ('regressor',
                 RandomForestRegressor(max_depth=6, max_features='sqrt',
                                       n_estimators=500))])

In [ ]:
y_pred2= pipe.predict(X_test)